In [1]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import cufflinks as cf
cf.go_offline()
import warnings 
warnings.filterwarnings('ignore')


import plotly as pl
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import sklearn

import sklearn.linear_model as linear_model
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from IPython.display import HTML, display


In [2]:
data=pd.read_csv('BlackFriday.csv')

In [3]:
features=data.columns.values

In [4]:
features

array(['User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation',
       'City_Category', 'Stay_In_Current_City_Years', 'Marital_Status',
       'Product_Category_1', 'Product_Category_2', 'Product_Category_3',
       'Purchase'], dtype=object)

In [5]:
data.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537577 entries, 0 to 537576
Data columns (total 12 columns):
User_ID                       537577 non-null int64
Product_ID                    537577 non-null object
Gender                        537577 non-null object
Age                           537577 non-null object
Occupation                    537577 non-null int64
City_Category                 537577 non-null object
Stay_In_Current_City_Years    537577 non-null object
Marital_Status                537577 non-null int64
Product_Category_1            537577 non-null int64
Product_Category_2            370591 non-null float64
Product_Category_3            164278 non-null float64
Purchase                      537577 non-null int64
dtypes: float64(2), int64(5), object(5)
memory usage: 49.2+ MB


In [7]:
#Let us check the null values and it's percentage

temp_df=data.isnull().sum().reset_index()
temp_df['Percentage of Null Values']= temp_df[0]/len(data)
temp_df.columns=['Column Nmae','Number of Null values','Percentage of Null values']
temp_df

,Column Nmae,Number of Null values,Percentage of Null values
0,User_ID,0,0.000000
1,Product_ID,0,0.000000
2,Gender,0,0.000000
3,Age,0,0.000000
4,Occupation,0,0.000000
5,City_Category,0,0.000000
6,Stay_In_Current_City_Years,0,0.000000
7,Marital_Status,0,0.000000
8,Product_Category_1,0,0.000000
9,Product_Category_2,166986,0.310627


In [8]:
#remove product_category_3
data.drop(labels='Product_Category_3',axis=1,inplace=True)

In [9]:
new_df=data.Product_Category_2.value_counts().reset_index()

In [10]:
new_df.iplot(kind='bar',x='index',y='Product_Category_2')

In [11]:
#fillnavalues
data['Product_Category_2'].fillna(value=8,inplace=True)
data['Product_Category_2'].unique()

array([ 8.,  6., 14.,  2., 15., 16., 11.,  5.,  3.,  4., 12.,  9., 10.,
       17., 13.,  7., 18.])

In [12]:
#Ratio of Male and female
gender=data['Gender'].value_counts().reset_index()
gender.columns=['Gender','Count']

In [13]:
gender.iplot(kind='pie', labels='Gender', values='Count', hole=0.2, pull=0.2, title='Ratio of Male and Female')

In [14]:
#agecounts
age =data['Age'].value_counts().reset_index()
age.columns=['Age','Count']
age.iplot(kind='pie',labels='Age',values='Count',pull=0.2)

In [15]:
#remove the useless columns

data.drop(['User_ID','Product_ID'],axis=1,inplace=True)

In [16]:
data.head()

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Purchase
0,F,0-17,10,A,2,0,3,8.0,8370
1,F,0-17,10,A,2,0,1,6.0,15200
2,F,0-17,10,A,2,0,12,8.0,1422
3,F,0-17,10,A,2,0,12,14.0,1057
4,M,55+,16,C,4+,0,8,8.0,7969


In [17]:
#city
city=data['City_Category'].value_counts().reset_index()
city.columns=['Cities','Count']

In [18]:
city.iplot(kind='pie',labels='Cities',values='Count')

In [19]:
#Total purchase amount of male and female
data.head()

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Purchase
0,F,0-17,10,A,2,0,3,8.0,8370
1,F,0-17,10,A,2,0,1,6.0,15200
2,F,0-17,10,A,2,0,12,8.0,1422
3,F,0-17,10,A,2,0,12,14.0,1057
4,M,55+,16,C,4+,0,8,8.0,7969


In [20]:
data.groupby(['Gender']).agg({'Purchase':sum}).iplot(kind='bar')

In [21]:
#Ratio of married and non-married populations

data.head()

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Purchase
0,F,0-17,10,A,2,0,3,8.0,8370
1,F,0-17,10,A,2,0,1,6.0,15200
2,F,0-17,10,A,2,0,12,8.0,1422
3,F,0-17,10,A,2,0,12,14.0,1057
4,M,55+,16,C,4+,0,8,8.0,7969


In [22]:
data['Marital_Status'].value_counts().reset_index().iplot(kind='pie',labels='index',values='Marital_Status')

In [23]:
#purchase amount of married and unmarried people
data.groupby(['Marital_Status']).agg({'Purchase':sum}).reset_index().iplot(kind='pie',labels='Marital_Status',values='Purchase')

In [24]:
data.head()

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Purchase
0,F,0-17,10,A,2,0,3,8.0,8370
1,F,0-17,10,A,2,0,1,6.0,15200
2,F,0-17,10,A,2,0,12,8.0,1422
3,F,0-17,10,A,2,0,12,14.0,1057
4,M,55+,16,C,4+,0,8,8.0,7969


In [25]:
pro1=data['Product_Category_1'].reset_index()
pro1.columns=['index','Product']

In [26]:
pro2=data['Product_Category_2'].reset_index()
pro2.columns=['index','Product']

In [27]:
product=pd.concat([pro1,pro2])

In [28]:
product=product['Product'].value_counts().reset_index()

In [29]:
product.iplot(kind='bar',x='index',y='Product')

In [30]:
#rich city among ABC
data.head()

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Purchase
0,F,0-17,10,A,2,0,3,8.0,8370
1,F,0-17,10,A,2,0,1,6.0,15200
2,F,0-17,10,A,2,0,12,8.0,1422
3,F,0-17,10,A,2,0,12,14.0,1057
4,M,55+,16,C,4+,0,8,8.0,7969


In [31]:
data.groupby(['City_Category']).agg({'Purchase':sum}).reset_index().iplot(kind='pie',labels='City_Category',values='Purchase')

In [32]:
#Regression

from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.svm import SVR,SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA

D:\anaconda\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning:

numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.



In [33]:
data.head()

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Purchase
0,F,0-17,10,A,2,0,3,8.0,8370
1,F,0-17,10,A,2,0,1,6.0,15200
2,F,0-17,10,A,2,0,12,8.0,1422
3,F,0-17,10,A,2,0,12,14.0,1057
4,M,55+,16,C,4+,0,8,8.0,7969


In [34]:
data.head()
X_train=data.iloc[:,:-1].values
y_train=data.iloc[:,-1].values

In [35]:
#head 
#dealing with categorical values

# Apply LabelEncoder to convert categorical values into 0,1 and so on format.
#[0,1,3,4]
label_x =LabelEncoder()
X_train[:,0]=label_x.fit_transform(X_train[:,0])
X_train[:,1]=label_x.fit_transform(X_train[:,1])
X_train[:,3]=label_x.fit_transform(X_train[:,3])
X_train[:,4]=label_x.fit_transform(X_train[:,4])



In [36]:
#Apply onehot encoder to split those columns who have more then 2 categories

one_hot=OneHotEncoder(categorical_features=[1,2,3,4,6,7])
X_train=one_hot.fit_transform(X_train).toarray()

In [37]:
print(X_train.shape)

(537577, 73)


In [38]:
#Apply PCA to reduce the number of columns 

# Scaling the feature matrix.
sc_x = StandardScaler()
X_train = sc_x.fit_transform(X_train)

# Scaling the Target Vector.
sc_y = StandardScaler()
y_train = y_train.reshape(-1,1)    # Converting the 1-D array into 2-array.
y_train = sc_x.fit_transform(y_train)

In [39]:
#PCA

pca = PCA(n_components=None)
X_train = pca.fit_transform(X_train)
pca.explained_variance_ratio_

array([3.07961958e-02, 2.80003491e-02, 2.66756054e-02, 2.54718396e-02,
       2.33202625e-02, 2.25407747e-02, 2.13145111e-02, 2.10317684e-02,
       2.04429437e-02, 2.00029285e-02, 1.89966429e-02, 1.87807791e-02,
       1.84106864e-02, 1.81799034e-02, 1.75751546e-02, 1.74209028e-02,
       1.72303758e-02, 1.70204837e-02, 1.68322704e-02, 1.64045044e-02,
       1.63145089e-02, 1.58948567e-02, 1.57087607e-02, 1.51043525e-02,
       1.49086623e-02, 1.48346772e-02, 1.46003913e-02, 1.45148249e-02,
       1.44051786e-02, 1.43241345e-02, 1.42060894e-02, 1.41638065e-02,
       1.41516376e-02, 1.41276351e-02, 1.40778659e-02, 1.40248895e-02,
       1.39768981e-02, 1.39594325e-02, 1.39078744e-02, 1.38717463e-02,
       1.37867399e-02, 1.37354140e-02, 1.37154520e-02, 1.37061044e-02,
       1.36595277e-02, 1.36092734e-02, 1.35638587e-02, 1.34920774e-02,
       1.33647545e-02, 1.32708359e-02, 1.31506711e-02, 1.28339284e-02,
       1.26882779e-02, 1.24630729e-02, 1.18506095e-02, 1.07824242e-02,
      

In [40]:
#taking number of features e-02
pca = PCA(n_components=50)   # Fitting PCA with 50 features.
X_train = pca.fit_transform(X_train)

In [ ]:
# Multi-linear regression Model. 
regressor_multi = LinearRegression()
regressor_multi.fit(X_train,y_train)
# Let us check the accuray
accuracy = cross_val_score(estimator=regressor_multi, X=X_train, y=y_train,cv=10)
print(f"The accuracy of the Multi-linear Regressor Model is \t {accuracy.mean()}")
print(f"The deviation in the accuracy is \t {accuracy.std()}")
print()